# Make heatmap for Lipidds

## Include libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


from matplotlib import cm
from matplotlib import pylab as plt
from matplotlib.lines import Line2D
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import seaborn as sns

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Functions and definitions

In [2]:
def make_colormap(seq):
    """Return a LinearSegmentedColormap
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
    and in the interval (0,1).
    """
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

#Colormap suggested by Julia
c = mcolors.ColorConverter().to_rgb
Julia_cmap = make_colormap(
    [c('#f6e1d1'), c('#941852')])

In [3]:
#define classes of lipids e.g. PC = Phosphatidylcholines
types_of_Lipids = ['CE','Cer','DAG','LPC','LPE','PC','PE','PI','PS','SM','TAG']

#colormap (20 unique colors)
cmap = cm.get_cmap('tab20')

#assign for each class of lipid a unique color
lipid_color = {}
for i,l in enumerate(types_of_Lipids):
    lipid_color[l] = cmap(i)
    


### 1. Read file

In [8]:
#Considered outlier
outlier = ['1 FBC','9 FBC 206']

#read file
fp = open('../data/Normalized_lipid_counts_0200522.csv')
lipids = fp.readline().strip().split(',')[6:]

#read all rows
sample_results = {}
for line in fp:
    tmp = line.strip().split(',')
    
    
    
    lipid_values = []
    for val in tmp[6:]:
        if val == '#VALUE!':
            lipid_values.append(0)
        else:
            lipid_values.append(float(val))
    
    if tmp[2] not in outlier:
        sample_results[tmp[2]] = lipid_values

print('Number of samples: %d' %len(sample_results))
print('Number of lipids per sample: %d' %len(sample_results['2 FBC']))

Number of samples: 8
Number of lipids per sample: 284


### 2. Make Clustermap for all lipids

In [10]:

#make a data vector that can be later used to plot a heatmap
data = []
sample_labels = []
for key in sample_results:
    data.append(sample_results[key])
    sample_labels.append(key)


#Transpose data so that samples = columns, lipids = rows
transpose = np.array(data).T.tolist()


#Go through all rows, if a lipid has no entry (measurement) for one of the sample; remove it
#Add a row color (corresponding to the lipid class)
final_rows = []
row_cols = []
for row,l in zip(transpose,lipids):
    if row.count(0) != len(sample_results.keys()):
        
        #extract lipid color
        lipid_group = l.split(' ')[0]
        #add row color
        row_cols.append(lipid_color[lipid_group])
        #final valid lipid
        final_rows.append(row)

print("Invalid lipids: %d" %(len(transpose) - len(final_rows)))
print('Final number of valid lipids: %d' %len(final_rows))

# Make cluster map and save
##
sns.clustermap(final_rows,z_score=0,method='median',
               col_colors=['#3AB9D1','#3AB9D1','#3AB9D1','#3AB9D1','#F8B301','#F8B301','#F8B301','#F8B301'],
               row_colors=row_cols,xticklabels=sample_labels,yticklabels=[],
               cmap=Julia_cmap)
plt.ylabel('Z-Score')
plt.savefig('../results/Lipidomics_Heatmap/All_Lipids.pdf')
plt.close()

Invalid lipids: 59
Final number of valid lipids: 225


### 3. Make vulcano plot for all lipids (between FBC and FBC206)

In [53]:
# Make volcano plot showing FC and PValue of FBC206 vs. FBC
# final_rows[0-3] are FBC while [3-5] are FBC206
##

data_vulcano = []
log2FC = []
significance = []
for row in transpose:
    if row.count(0) != len(sample_results.keys()):
        FBC_Values = row[0:4]
        FBC206_Values = row[4:8]


        #calculate the statistics (see before)
        significance.append(ttest_ind(FBC206_Values,FBC_Values)[1])
        log2FC.append(np.mean(FBC206_Values)/np.mean(FBC_Values))
  

# correct pValues according to benjamini hochberg (FDR)
pValues_Corrected =  multipletests(significance,alpha=0.05,method='fdr_bh')[1]
pValues_Corrected = [-np.log10(p) for p in pValues_Corrected]

#transform the foldchanges to log2
foldchanges = [np.log2(f) for f in log2FC]



####
# START MAKING PLOT FBC vs. FBC206
# Scatter plot showing for each individual lipid the foldchange (x-axis) and pvalue (y-axis)
####

# Make result plot for differences upon ACM treatment (for wildtype)
plt.title('FBC / FBC206')
plt.scatter(foldchanges,pValues_Corrected,c=row_cols, alpha = 0.4)
plt.axhline(-np.log10(0.05), color= 'grey', ls='--')
plt.xlabel('log2[Fold Change FBC206/FBC]')
plt.ylabel('-log[PValue]')

# Make a legend showing the colors to lipid classes
legend_elements = []
for key in lipid_color:
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label=key,
                      markerfacecolor=lipid_color[key], markersize=10))

# Make actual plot
plt.legend(handles=legend_elements, loc='upper right',prop={'size': 5}, frameon=False)
plt.savefig('../results/Lipidomics_Heatmap/Vulcano_AllLipids.pdf')
plt.close()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


### 4. Group lipids into lipid groups

In [20]:
# Create empty dictionary that contains final results per lipid group
##
final_rows = []
lipid_group_results = {}
for l in types_of_Lipids:
    lipid_group_results[l] = []

# Go through all values in original vector, and assign the values to the correct lipid groups
##
for row,l in zip(transpose,lipids):
    if row.count(0) != len(sample_results.keys()):
        lipid_group = l.split(' ')[0]
        lipid_group_results[lipid_group].append(row)


# For each lipid group calculate the mean of all associated lipids within the group
# Also add row color depending on the the lipid class
##
final_rows = []
row_cols = []
for group in lipid_group_results:
    
    #add row color (=lipid class)
    row_cols.append(lipid_color[group])

    #add lipid group (=class) mean
    vals = np.mean(lipid_group_results[group], axis=0)
    
    #add the calculated mean value to final rows
    final_rows.append(vals)

# Make a clustermap
##
g = sns.clustermap(final_rows,z_score=0,method='complete', col_colors=['#3AB9D1','#3AB9D1','#3AB9D1','#3AB9D1','#F8B301','#F8B301','#F8B301','#F8B301'],
               row_colors=row_cols,xticklabels=sample_labels,yticklabels=[],
                  cmap=Julia_cmap)

plt.ylabel('Z-Score')
plt.savefig('../results/Lipidomics_Heatmap/Lipid_groups.pdf')
plt.close()



# Make an additional legend showing the colors to lipid classes
fig = plt.figure()
legend_elements = []
for key in lipid_color:
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label=key,
                      markerfacecolor=lipid_color[key], markersize=10))

# Make actual plot
plt.legend(handles=legend_elements, loc='upper right',prop={'size': 5}, frameon=False)
plt.savefig('../results/Lipidomics_Heatmap/Legend.pdf')
plt.close()

In [58]:
# Make volcano plot showing FC and PValue of FBC206 vs. FBC
# final_rows[0-3] are FBC while [3-5] are FBC206
##

data_vulcano = []

log2FC = []
significance = []
colors = []
for group in lipid_group_results:
    colors.append(lipid_color[group])
    #final_rows.append((np.mean(lipid_group_results[group], axis=0)))

    
    vals = np.mean(lipid_group_results[group], axis=0)
    
    FBC_Values = vals[0:4]
    FBC206_Values = vals[4:8]


    #calculate the statistics (see before)
    significance.append(ttest_ind(FBC206_Values,FBC_Values)[1])
    log2FC.append(np.mean(FBC206_Values)/np.mean(FBC_Values))
    
    
# correct pValues according to benjamini hochberg (FDR)
pValues_Corrected =  multipletests(significance,alpha=0.05,method='fdr_bh')[1]
pValues_Corrected = [-np.log10(p) for p in pValues_Corrected]

#transform the foldchanges to log2
foldchanges = [np.log2(f) for f in log2FC]




####
# START MAKING PLOT FBC vs. FBC206
# Scatter plot showing for each individual lipid the foldchange (x-axis) and pvalue (y-axis)
####

# Make result plot for differences upon ACM treatment (for wildtype)
plt.title('FBC / FBC206')
plt.scatter(foldchanges,pValues_Corrected,c=colors, alpha = 0.4)
plt.axhline(-np.log10(0.05), color= 'grey', ls='--')
plt.xlabel('log2[Fold Change FBC206/FBC]')
plt.ylabel('-log[PValue]')

# Make a legend showing the colors to lipid classes
legend_elements = []
for key in lipid_color:
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label=key,
                      markerfacecolor=lipid_color[key], markersize=10))

# Make actual plot
plt.legend(handles=legend_elements, loc='upper right',prop={'size': 5}, frameon=False)
plt.savefig('../results/Lipidomics_Heatmap/Vulcano_Lipidgroups.pdf')
plt.close()